In [1]:
import ee
import geemap
import os
import shapely
import json
import mgrs
import pandas as pd

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map()

In [4]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [5]:
def import_aois(csv_loc):    

        df_labels = pd.read_csv(csv_loc)
        df_labels = df_labels[["center-lat","center-long","polygon","Labels combined"]]

        polygons = []
        for polygon in df_labels["polygon"]:
            polygons.append(json.loads(polygon)["coordinates"])


        tiles = []
        tiles_dic = {}
        polygon_id = 0 
        coordinates = []
        m = mgrs.MGRS()
        for items in polygons:
            polygon_id += 1 
            for item in items:
                for lon_lat in item:
                    coordinates.append(lon_lat)
                    c = m.toMGRS(lon_lat[1], lon_lat[0])
                    tile = c[0:5]
                    

                    if polygon_id in tiles_dic:

                        tiles_dic[polygon_id].append(tile)

                    else:

                        tiles_dic[polygon_id] = [tile]

                    tiles.append(tile)

                tiles_dic[polygon_id] = list(set(tiles_dic[polygon_id]))

        tiles = list(set(tiles))

        df_labels["tiles"] = tiles_dic.values()

        #bounding box

        min_lon = min([i[0] for i in coordinates])
        min_lat = min([i[1] for i in coordinates])
        max_lon = max([i[0] for i in coordinates])
        max_lat = max([i[1] for i in coordinates])

        bounding_box = min_lon,min_lat,max_lon,max_lat
        
        return polygons

In [6]:
polygons = import_aois("/Volumes/Lacie/zhenyadata/Project_Canopy_Data/PC_Data/Sentinel_Data/Labelled/Tiles_v3/Polygon_List/polygons_101320.csv")

In [7]:
polygons[0]

[[[9.088783, 5.753152],
  [9.179249, 5.703106],
  [9.248257, 5.792946],
  [9.236069, 5.845887],
  [9.090672, 5.844009],
  [9.088783, 5.753152]]]

In [8]:
# # region of interest
# geom = ee.Geometry.Polygon(polygons[0])



In [9]:
# help(ee.ImageCollection)

In [10]:
# # image collection
# ic = ee.ImageCollection('COPERNICUS/S2')\
#     .filterBounds(geom)\
#     .filterDate(startDate,stopDate)
    
# # # single image
# # img = ee.Image(ic.first())

In [11]:
# ee.Image(ic.get(0)).getInfo()

In [21]:
print(polygons[0])

# start and end of time series
startDate = ee.Date('2019-01-01')
stopDate  = ee.Date('2020-12-31')

# create an roi. first item in Misha's label list
polys = ee.Geometry.Polygon(polygons[0])

# Add Earth Engine dataset
# Sentinel 2 ImageCollection
# filter by bounds and data
collection = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(polys)\
    .filterDate(startDate,stopDate)\
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE",5))\
    .sort('CLOUD_COVER',True)
# # create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# filter the ImageCollection using the roi
mosaic = collection.mosaic()

# # # print out the number of images in the ImageCollection
count = collection.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
vis = {'bands': ["B4", "B3", "B2"]}
Map.addLayer(mosaic,vis)
Map.addLayer(fc)

# export the ImageCollection to Google Drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'image2'}  # scale means resolution.
img_lst = collection.toList(100)

for i in range(0, 1):
    image = ee.Image(img_lst.get(i))
    image = image.select('B.+')
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image, name, downConfig)
    task.start()
    print(task.status())

[[[9.088783, 5.753152], [9.179249, 5.703106], [9.248257, 5.792946], [9.236069, 5.845887], [9.090672, 5.844009], [9.088783, 5.753152]]]
lng = 9.163954053102769, lat = 5.7876496002257225
Count:  14
20190110T094359_20190110T095700_T32NNM
{'state': 'READY', 'description': '20190110T094359_20190110T095700_T32NNM', 'creation_timestamp_ms': 1603833604487, 'update_timestamp_ms': 1603833604487, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'YNKCTAWQ3DQCWFFEWWYU734T', 'name': 'projects/earthengine-legacy/operations/YNKCTAWQ3DQCWFFEWWYU734T'}


In [30]:
while task.status()["state"] == 'RUNNING':
    print(task.status())

{'state': 'RUNNING',
 'description': '20190110T094359_20190110T095700_T32NNM',
 'creation_timestamp_ms': 1603833604487,
 'update_timestamp_ms': 1603833765543,
 'start_timestamp_ms': 1603833615000,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'YNKCTAWQ3DQCWFFEWWYU734T',
 'name': 'projects/earthengine-legacy/operations/YNKCTAWQ3DQCWFFEWWYU734T'}

In [119]:
help(task)

Help on Task in module ee.batch object:

class Task(builtins.object)
 |  Task(task_id, task_type, state, config=None, name=None)
 |  
 |  A batch task that can be run on the EE batch processing system.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, task_id, task_type, state, config=None, name=None)
 |      Creates a Task with the given ID and configuration.
 |      
 |      The constructor is not for public use. Instances can be obtained by:
 |      - Calling the static method Task.list().
 |      - Calling any of the methods on the Export static class.
 |      - Unpickling a previously pickled Task object.
 |      
 |      If you're looking for a task's status but don't need a full task object,
 |      ee.data.getTaskStatus() may be appropriate.
 |      
 |      Args:
 |        task_id: The task ID, originally obtained through ee.data.newTaskId().
 |          May be None if the ID is not yet known.
 |        task_type: The type of the task; one of the values in Task.Type.
 |  

In [89]:
help(image.get)

Help on method Element.get in Element:

Element.get(*args, **kwargs) method of ee.image.Image instance
    Extract a property from a feature.
    
    Args:
      object: The feature to extract the property from.
      property: The property to extract.



In [97]:
# Add Earth Engine dataset
# USDA NAIP ImageCollection
collection = ee.ImageCollection('COPERNICUS/S2')

# create an roi
polys = ee.Geometry.Polygon(
        [[[-99.29615020751953, 46.725459351792374],
          [-99.2116928100586, 46.72404725733022],
          [-99.21443939208984, 46.772037733479884],
          [-99.30267333984375, 46.77321343419932]]])

# create a FeatureCollection based on the roi and center the map
centroid = polys.centroid()
lng, lat = centroid.getInfo()['coordinates']
print("lng = {}, lat = {}".format(lng, lat))
Map.setCenter(lng, lat, 12)
fc = ee.FeatureCollection(polys)

# filter the ImageCollection using the roi
naip = collection.filterBounds(polys)
naip_2015 = naip.filterDate('2020-01-01', '2020-12-31')
mosaic = naip_2015.mosaic()

# print out the number of images in the ImageCollection
count = naip_2015.size().getInfo()
print("Count: ", count)

# add the ImageCollection and the roi to the map
vis = {'bands': ["B4", "B3", "B2"]}
Map.addLayer(mosaic,vis)
Map.addLayer(fc)

# export the ImageCollection to Google Drive
downConfig = {'scale': 30, "maxPixels": 1.0E13, 'driveFolder': 'image'}  # scale means resolution.
img_lst = naip_2015.toList(100)

for i in range(0, count):
    image = ee.Image(img_lst.get(i))
    # name = image.get('system:time_start').getInfo()
    # name = image.get('CLOUDY_PIXEL_PERCENTAGE').getInfo()
    name = image.get('system:index').getInfo()
    print(name)
    task = ee.batch.Export.image(image, name, downConfig)
    task.start()

lng = -99.25621066337796, lat = 46.748869171015414
Count:  121
20200101T173721_20200101T173724_T14TMS


KeyboardInterrupt: 

In [19]:
help(image.getInfo()))

TypeError: getInfo() takes 1 positional argument but 2 were given